In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [3]:
# Extract
file = "SARS_2003.csv"
sars = pd.read_csv(file)
sars.head()

,Date,Country,Cumulative number of case(s),Number of deaths,Number recovered
0,2003,Germany,1,0,0
1,2003,Canada,8,2,0
2,2003,Singapore,20,0,0
3,2003,"Hong Kong SAR, China",95,1,0
4,2003,Switzerland,2,0,0


In [7]:
sars = sars[['Date', 'Country', 'Cumulative number of case(s)', 'Number of deaths']].copy()
# sars.head()

In [27]:
# Rename the column headers
sars_clean = sars.rename(columns={"Date": "year_sars",
                                  "Country": "country",
                                 "Cumulative number of case(s)": "number_of_cases_sars",
                                 "Number of deaths": "number_of_deaths_sars"})
sars_clean.head()

,year_sars,country,number_of_cases_sars,number_of_deaths_sars
0,2003,Germany,1,0
1,2003,Canada,8,2
2,2003,Singapore,20,0
3,2003,"Hong Kong SAR, China",95,1
4,2003,Switzerland,2,0


In [17]:
# Extract
file1 = "TB_2007.csv"
tb = pd.read_csv(file1)
tb = tb[['Year','Country', 'Number of prevalent tuberculosis cases', 'Deaths due to tuberculosis among HIV-negative people (per 100 000 population)']].copy()

In [18]:
tb['Number of prevalent tuberculosis cases'] = tb['Number of prevalent tuberculosis cases'].apply(lambda x:x.replace(' ','')).astype(float)
tb.head()

,Year,Country,Number of prevalent tuberculosis cases,Deaths due to tuberculosis among HIV-negative people (per 100 000 population)
0,2014,Afghanistan,110000.0,44.0
1,2013,Afghanistan,100000.0,42.0
2,2012,Afghanistan,98000.0,40.0
3,2011,Afghanistan,94000.0,40.0
4,2010,Afghanistan,91000.0,39.0


In [25]:
tb_clean = tb.rename(columns={"Year":"year_tb",
                              "Country": "country",
                                 "Number of prevalent tuberculosis cases": "number_of_cases_tb",
                                 "Deaths due to tuberculosis among HIV-negative people (per 100 000 population)": "number_of_deaths_tb"})
tb_clean.head()

,year_tb,country,number_of_cases_tb,number_of_deaths_tb
0,2014,Afghanistan,110000.0,44.0
1,2013,Afghanistan,100000.0,42.0
2,2012,Afghanistan,98000.0,40.0
3,2011,Afghanistan,94000.0,40.0
4,2010,Afghanistan,91000.0,39.0


In [20]:
# Extract
file2 = "HN_2009.csv"
hn = pd.read_csv(file2, encoding= 'latin1')
# hn.head()

In [21]:
# Transform
hn = hn[['Update Time','Country', 'Cases','Deaths',]]

In [26]:
# Transform
hn_clean = hn.rename(columns={ "Update Time":"year_h1n1",
                                "Country": "country",
                                 "Cases": "number_of_cases_h1n1",
                                 "Deaths": "number_of_deaths_h1n1",
                             })
hn_clean.head()

,year_h1n1,country,number_of_cases_h1n1,number_of_deaths_h1n1
0,2009,Algeria,5,0.0
1,2009,Antigua and Barbuda,2,0.0
2,2009,Argentina,2485,60.0
3,2009,Australia,5298,10.0
4,2009,Austria,19,0.0


In [28]:
# Merge the dataframes to create one large df
# Transform
df_1 = pd.merge(sars_clean,tb_clean, on='country')
# df_1.head()

df_new = pd.merge(df_1,hn_clean, on='country')
# df_new.set_index('country', inplace=True)

In [29]:
df_f = df_new.sort_values('country', ascending=True)
#df_f.reset_index(inplace=True)
# df_f.drop(columns=['Index'], axis=1, inplace=True)
df_f.head()

,year_sars,country,number_of_cases_sars,number_of_deaths_sars,year_tb,number_of_cases_tb,number_of_deaths_tb,year_h1n1,number_of_cases_h1n1,number_of_deaths_h1n1
173025,2003,Australia,3,0,2014,1900.0,0.18,2009,4568,9.0
172619,2003,Australia,1,0,2009,1700.0,0.20,2009,1051,0.0
172618,2003,Australia,1,0,2009,1700.0,0.20,2009,1224,0.0
172617,2003,Australia,1,0,2009,1700.0,0.20,2009,1307,0.0
172616,2003,Australia,1,0,2009,1700.0,0.20,2009,1307,0.0


In [33]:
# Load

connection_string = "postgres:postgres@localhost:5433/pandemics"
engine = create_engine(f'postgresql://{connection_string}')

In [34]:
engine.table_names()

['pandemic_data']

In [37]:
df_f.to_sql(name='pandemic_data', con=engine, if_exists='append', index=False)

In [38]:
pd.read_sql_query('select * from pandemic_data', con=engine).head()

,country,year_sars,number_of_cases_sars,number_of_deaths_sars,year_tb,number_of_cases_tb,number_of_deaths_tb,year_h1n1,number_of_cases_h1n1,number_of_deaths_h1n1
0,Australia,2003,3,0,2014,1900.0,0.18,2009,4568,9.0
1,Australia,2003,1,0,2009,1700.0,0.2,2009,1051,0.0
2,Australia,2003,1,0,2009,1700.0,0.2,2009,1224,0.0
3,Australia,2003,1,0,2009,1700.0,0.2,2009,1307,0.0
4,Australia,2003,1,0,2009,1700.0,0.2,2009,1307,0.0
